In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import tensorflow_addons as tfa
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers

import pylab as pl

sys.path.append('../input/swintransformertf')
from swintransformer import SwinTransformer

In [ ]:
tf.random.set_seed(0)
np.random.seed(0)

In [ ]:
RES = [384, 384]
META_SHAPE = []
BATCH_SIZE = 16
BINS = 30
FOLDS = 5
EPOCHS = 5
AUGMENT = True
SEED = 2022

In [ ]:
# See https://www.kaggle.com/cdeotte/cutmix-and-mixup-on-gpu-tpu
def mixup(inputs, label, PROBABILITY=0.5):
    # input image - is a batch of images of size [n,dim,dim,3] not a single image of [dim,dim,3]
    # output - a batch of images with mixup applied
    DIM = RES[0]
    CLASSES = 1
    
    image = inputs['image_inp']
    
    batch_size = BATCH_SIZE
    
    imgs = []; labs = []
    for j in range(batch_size):
        # DO MIXUP WITH PROBABILITY DEFINED ABOVE
        P = tf.cast( tf.random.uniform([],0,1)<=PROBABILITY, tf.float32)
        # CHOOSE RANDOM
        k = tf.cast( tf.random.uniform([],0,batch_size),tf.int32)
        a = tf.random.uniform([],0,1)*P # this is beta dist with alpha=1.0
        # MAKE MIXUP IMAGE
        img1 = image[j,]
        img2 = image[k,]
        imgs.append((1-a)*img1 + a*img2)
        
        lab1 = label[j,]
        lab2 = label[k,]
        labs.append((1-a)*lab1 + a*lab2)
            
    # RESHAPE HACK SO TPU COMPILER KNOWS SHAPE OF OUTPUT TENSOR (maybe use Python typing instead?)
    image2 = tf.reshape(tf.stack(imgs),(BATCH_SIZE,DIM,DIM,3))
    label2 = tf.reshape(tf.stack(labs),(BATCH_SIZE,CLASSES))
    return ({ 'image_inp': image2, 'meta_inp': inputs['meta_inp'] }, label2)

In [ ]:
train_base_path = "../input/petfinder-pawpularity-score/train/"
train_csv = pd.read_csv("../input/petfinder-pawpularity-score/train.csv")
train_data = np.array(train_csv)

binned = [[] for x in range(BINS)]

for datum in train_data:
    datum[0] = train_base_path+datum[0]+".jpg"
    binned[min(BINS-1, int(datum[-1]//(100/BINS)))].append(datum)
    
train_images = train_data[:,0]
train_meta = train_data[:,1:-1].astype(np.float32)
train_labels = train_data[:,-1].astype(np.float32) / 100.

FOLD_SIZE = train_images.shape[0] // FOLDS
print(f"Fold size: {FOLD_SIZE}")
train_datasets = []
val_datasets = []

def get_preprocess_fn(training):
    if training:
        return preprocess_train
    
    return preprocess_val

def preprocess_general(inputs):
    filename = inputs['image_inp']
    image = tf.io.read_file(filename)
    image = tf.io.decode_jpeg(image)
    #image = tf.image.convert_image_dtype(image, tf.float32)
    #image = tf.image.per_image_standardization(image)
    image = tf.image.resize(image, RES)
    image = keras.applications.imagenet_utils.preprocess_input(image, mode='torch')
    
    return image

def preprocess_train(inputs, labels):
    image = preprocess_general(inputs)

    if AUGMENT:
        image = tf.image.random_flip_left_right(image, SEED)
        image = tf.image.random_flip_up_down(image, SEED)
        image = tf.image.random_hue(image, 0.05, SEED)
        image = tf.image.random_contrast(image, 0.95, 1.05, SEED)
        image = tf.image.random_brightness(image, 0.05, SEED)

    return ({ 'image_inp': image, 'meta_inp': inputs['meta_inp'] }, labels)

def preprocess_val(inputs, labels):
    image = preprocess_general(inputs)

    return ({ 'image_inp': image, 'meta_inp': inputs['meta_inp'] }, labels)

total_trainset = tf.data.Dataset.from_tensor_slices(
        (
            { 'image_inp': train_images, 'meta_inp': train_meta },
            { 'output': train_labels }
        )
    ).map(get_preprocess_fn(True)).shuffle(1024).batch(BATCH_SIZE)

# Returns train, val
def extract_validation_set(data, fold):
    if FOLDS == 1:
        return data[:int(0.8*len(data))], data[int(0.8*len(data)):]
    
    return np.concatenate((data[:fold*FOLD_SIZE], data[(fold+1)*FOLD_SIZE:]), axis=0), data[fold*FOLD_SIZE:(fold+1)*FOLD_SIZE]

for f in range(FOLDS):
    train_images_fold, val_images_fold = extract_validation_set(train_images, f)
    train_meta_fold, val_meta_fold = extract_validation_set(train_meta, f)
    train_labels_fold, val_labels_fold = extract_validation_set(train_labels, f)

    META_SHAPE = train_meta[0].shape

    train_datasets.append(
        tf.data.Dataset.from_tensor_slices(
            (
                { 'image_inp': train_images_fold, 'meta_inp': train_meta_fold },
                train_labels_fold
            )
        ).map(get_preprocess_fn(True)).shuffle(256).batch(BATCH_SIZE, drop_remainder=True)#.map(mixup)
    )
    
    val_datasets.append(
        tf.data.Dataset.from_tensor_slices(
            (
                { 'image_inp': val_images_fold, 'meta_inp': val_meta_fold },
                val_labels_fold
            )
        ).map(get_preprocess_fn(False)).shuffle(256).batch(BATCH_SIZE)
    )

In [ ]:
test_base_path = "../input/petfinder-pawpularity-score/test/"
test_csv = pd.read_csv("../input/petfinder-pawpularity-score/test.csv")
test_ids = list(test_csv['Id'])
test_meta = np.array(test_csv.drop('Id', axis=1)).astype(np.float32)

def test_preprocess(input_dict):
    image = preprocess_general(input_dict)
    
    return { 'image_inp': image, 'meta_inp': input_dict['meta_inp'] }

test_files = [test_base_path+s+".jpg" for s in test_ids]
test_dataset = tf.data.Dataset.from_tensor_slices({ 'image_inp': test_files, 'meta_inp': test_meta }).map(test_preprocess).batch(BATCH_SIZE)

In [ ]:
RES = [RES[0], RES[1], 3]

model_name = "swin_tiny_224"
#backbone = SwinTransformer(model_name, 
#                           num_classes=1, 
#                           include_top=False, 
#                           pretrained=False)

backbone = keras.applications.EfficientNetB2(include_top=False, weights=None, pooling='avg')

def get_model():
    image_inp = keras.Input(RES, name="image_inp")
    meta_inp = keras.Input(META_SHAPE, name="meta_inp")
    
    #backbone.load_weights(f"../input/swin-weights/{model_name}/{model_name}.ckpt")
    backbone.load_weights("../input/keras-applications-models/EfficientNetB2.h5")
    for layer in backbone.layers:
        if isinstance(layer, layers.BatchNormalization):
            layer.trainable = False
        else:
            layer.trainable = True
    
    #backbone.trainable = False
    
    x = backbone(image_inp)
    x = layers.Dropout(0.2)(x)
    #x = tf.concat([x, meta_inp], axis=-1)
    #x = layers.Dense(128, activation='gelu')(x)
    x = layers.Dense(1, name="output")(x)
    
    return keras.Model(inputs=[image_inp, meta_inp], outputs=x)

get_model()

def metric(labels, logits):
    las = labels * 100.
    los = tf.nn.sigmoid(logits) * 100.
    mse = tf.math.reduce_mean(tf.math.square((las - los)))
    rmse = tf.math.sqrt(mse)
    
    return rmse

def get_lr_callback():
    lr_start   = 0.000005
    lr_max     = 0.00000125 * BATCH_SIZE
    lr_min     = 0.000001
    lr_ramp_ep = 4
    lr_sus_ep  = 0
    lr_decay   = 0.8
   
    def lrfn(epoch):
        if epoch < lr_ramp_ep:
            lr = (lr_max - lr_start) / lr_ramp_ep * epoch + lr_start
            
        elif epoch < lr_ramp_ep + lr_sus_ep:
            lr = lr_max
            
        else:
            lr = (lr_max - lr_min) * lr_decay**(epoch - lr_ramp_ep - lr_sus_ep) + lr_min
            
        return lr

    lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=True)
    return lr_callback

In [ ]:
all_predictions = []

lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=5e-5,
    decay_steps=100, decay_rate=0.9,
    staircase=True)

opt = keras.optimizers.Adam(learning_rate=5e-6)

for fold in range(1):#fold, (train_dataset, val_dataset) in enumerate(zip(train_datasets, val_datasets)):
    keras.backend.clear_session()
    model = get_model()
    model.compile(optimizer=opt, loss=keras.losses.BinaryCrossentropy(from_logits=True), metrics=metric)
    print(f"TRAINING FOLD: {fold+1}")
    #model.fit(train_dataset, validation_data=val_dataset, epochs=EPOCHS)
    model.fit(total_trainset, epochs=EPOCHS)
    fold_predictions = model.predict(test_dataset)
    all_predictions.append(fold_predictions)
    
final_predictions = np.mean(all_predictions, axis=0)

In [ ]:
sub_dict = { 'Id': test_ids, 'Pawpularity': final_predictions[:,0] }
submission = pd.DataFrame.from_dict(sub_dict)

In [ ]:
submission.to_csv("submission.csv", index=False)
submission